In [ ]:
import json
import itertools
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.animation

from datetime import datetime, timedelta
from dateutil import rrule
from matplotlib import pyplot as plt
from matplotlib import dates as mdates

import IPython
from IPython.display import HTML

In [ ]:
%matplotlib inline

In [ ]:
sns.set_theme(style='white')

# Relative vs. Absolute Pitch Figure

In [ ]:
def make_packet(t, freq, t0, width=0.5):
    envelope = np.exp(-(t-t0)**2/(width**2))
    carrier = np.sin(2 * np.pi * freq * t)
    return carrier * envelope


This figure shows multiple waveforms at different frequencies. It's not easy to tell by eye what frequencies they are, but you can tell which ones are higher and lower frequency than one another.

In [ ]:
N = 2**16

ew = 1
hz_freqs = [2.5, 7.5, 15]
window =  6 * ew * len(hz_freqs)

t = np.linspace(0, window, N)
plt.figure(figsize=(16, 8))

for ii, freq in enumerate(hz_freqs):
    plt.plot(t, make_packet(t, freq, t0=(3 + 6 * ii) * ew, width=ew), lw=2.5)

plt.plot(t, [0] * len(t), 'k-', lw=2.5)

ax = plt.gca()
ax.set_yticks([])
ax.set_xlabel("Time (ms)");
plt.tight_layout()

This figure represents the three waveforms in the Fourier domain (plotting frequency vs. amplitude), where it is easy to see the absolute frequency of each wave.

In [ ]:
def do_fft(signal, convolution_kernel_size=0):
    output = np.real(np.sqrt(np.fft.rfft(signal)**2))
    if convolution_kernel_size > 0:
        kernel = np.ones(convolution_kernel_size) / convolution_kernel_size
        output = np.convolve(output, kernel, mode="same")
    return output

signal_width = window * 30

t2 = np.linspace(0, signal_width, N)
d = t2[1]-t2[0]
f = np.fft.rfftfreq(N, d=d)
ss = slice(0, np.where(f <= max(hz_freqs) + (0.75 * min(hz_freqs)))[0].max())

spectrum_max = 0
plt.figure(figsize=(16, 8))
for freq in hz_freqs:
    spectrum = do_fft(make_packet(t2, freq, t0=signal_width/2, width=ew))
    spectrum /= spectrum.sum()
    spectrum_max = max(spectrum_max, spectrum.max())
    plt.plot(f[ss], spectrum[ss], lw=5)

plt.plot(f[ss], [0] * len(f[ss]), '-k', lw=5)

ax = plt.gca()
ax.set_yticks([])
ax.set_xlabel("Frequency (kHz)");
ax.set_xlim(f[ss].min(), f[ss].max())
ax.set_ylim(0, spectrum_max.real * 1.1)
plt.tight_layout()

# Analysis of responses

In [ ]:
# Load the data from a history file
with open("data/cim_history.json") as f:
    json_data = json.load(f)

# Should only be one profile in here, so we'll extract that
assert len(json_data) == 1, "Need to select a profile in multi-profile histories"
json_data = next(iter(json_data.values()))

In [ ]:
COLORS = [
    "red",
    "yellow",
    "blue",
    "black",
    "green",
    "orange",
    "purple",
    "pink",
    "brown"
]

def to_idx(color):
    return COLORS.index(color)

In [ ]:
def set_date_formatter(ax):
    start = datetime.max
    end = datetime.min
    for line in ax.lines:
        x_data = line.get_xdata()
        x_min = min(x_data).replace(tzinfo=None)
        x_max = max(x_data).replace(tzinfo=None)
        start = min(start, x_min)
        end = max(end, x_max)
    
    if end - start < timedelta(days=120):
        rule = mdates.rrulewrapper(rrule.MONTHLY, bymonthday=range(1, 31, 3))
    elif end - start < timedelta(days=210):
        rule = mdates.rrulewrapper(rrule.MONTHLY, bymonthday=range(1,31,5))
    elif end - start < timedelta(days=365):
        rule = mdates.rrulewrapper(rrule.WEEKLY, bymonthday=1)
    else:
        rule = mdates.rrulewrapper(rrule.MONTHLY, bymonthday=1)

    xrange = (start, end)

    locator = mdates.RRuleLocator(rule)
    
    ax.set_xticks(ax.get_xticks(), labels=ax.get_xticklabels(), rotation=45)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(
        mdates.ConciseDateFormatter(
            ax.xaxis.get_major_locator(),
            show_offset=False,
            formats=[
                "%Y",
                r"$\bf{%b}$",
                "%d",
                "%H:%M",
                "%H:%M",
                "%S.%f",
            ],
        )
    )



In [ ]:
def get_session_history(history):
    all_sessions = []

    def to_datetime(df, col):
        return pd.to_datetime(df[col], unit="s", utc=True).dt.tz_convert("America/New_York")
    
    for color, data in history.items():
        if not data:
            continue
        session_df = pd.DataFrame(data)
        session_df = session_df[["current_chord", "start_time", "updated_time", "identifications", "correct"]]
        all_sessions.append(session_df)

    joined_df = pd.concat(all_sessions)
    joined_df = joined_df.rename({"current_chord": "color"}, axis=1)
    joined_df["start_time"] = to_datetime(joined_df, "start_time")
    joined_df["updated_time"] = to_datetime(joined_df, "updated_time")
    joined_df = joined_df.sort_values(by="start_time")
    joined_df["percent_correct"] = (joined_df["correct"] / joined_df["identifications"]) * 100
    return joined_df

In [ ]:
session_history = get_session_history(json_data)
session_history = session_history[session_history["color"] != "red"] # These are not interesting

In December 2023, we realized that we had moved on from Orange too quickly, because all through Purple and Pink, almost all the confusion was coming because he didn't have Orange down, so we jumped back to Orange. At the same time, we also switched to the "new" chords (using Tone.js), so I've divided the history into "old" and "new".

This history starts after we had already been using the program for a few months.

In [ ]:
switch_point = datetime(2023, 12, 9)
o_df = session_history.loc[session_history["start_time"].dt.tz_localize(None) < switch_point]
n_df = session_history.loc[session_history["start_time"].dt.tz_localize(None) >= switch_point]

o_df = o_df[o_df["color"] != "red"]

In [ ]:
def plot_session_history(df):
    fig = plt.figure(figsize=(10, 6))
    plt.plot(df["start_time"], df["percent_correct"].rolling(window=12).mean(), "-", zorder=0)
    plt.scatter(df["start_time"], df["percent_correct"], color=df["color"], zorder=1)
    set_date_formatter(plt.gca())
    plt.tight_layout()

plot_session_history(o_df)
plot_session_history(n_df)

In [ ]:
def make_confusion_matrix(history, color):
    history_size = COLORS.index(color) + 1
    sessions = history[color]
    arr = np.zeros((history_size, history_size, len(sessions)))
    for ii, session in enumerate(sessions):
        for target_color in session["confusion_matrix"]:
            for act_color, num in session["confusion_matrix"][target_color].items():
                arr[to_idx(target_color), to_idx(act_color), ii] = num
    return arr

In [ ]:
def replace_value(arr, value, new_value):
    arr_cp = arr.copy()
    arr_cp[arr_cp == value] = new_value
    return arr_cp

def normalize_cm(cm):
    return cm / replace_value(cm.sum(axis=1), 0, 1)

default_palette = sns.diverging_palette(10, 220, sep=10, n=20, as_cmap=True)

def plot_cm(cm, i, cmap=default_palette, subsample=1, ax=None):
    if ax is None:
        ax = plt.gca()

    tick_labels = COLORS[:cm.shape[0]]
    new_cm = normalize_cm(cm[:, :, i*subsample:(i + 1) * subsample].sum(axis=2))
    max_val = 1
    off_diagonal_mask = ~np.eye(cm.shape[0], dtype=bool)
    new_cm[off_diagonal_mask] = -new_cm[off_diagonal_mask]
    hm = sns.heatmap(new_cm[::-1,:],
                cmap=cmap,
                vmax=max_val,
                vmin=-max_val,
                xticklabels=tick_labels,
                yticklabels=tick_labels[::-1],
                cbar=False,
                ax=ax)
    ax.set_title(f"{i}")

    ax.set_xlabel("Guess")
    ax.set_ylabel("Correct")

    return hm

def animated_cm(cm, subsample = 10):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.gca()
    def animate(i):
        ax.clear()
        plot_cm(cm, i, subsample=subsample, ax=ax)
    animation = matplotlib.animation.FuncAnimation(fig, animate, frames=cm.shape[2] // subsample)
    plt.close()  # After the animation is done we don't need the plot anymore
    return animation
    

# Show the animation
HTML(animated_cm(make_confusion_matrix(json_data, "purple")).to_jshtml())

# Adaptive Mode Figures

In [ ]:
def matrix_to_json(arr):
    out = {}
    for i, ci in zip(range(arr.shape[0]), COLORS):
        out[ci] = {}
        for j, cj in zip(range(arr.shape[0]), COLORS):
            out[ci][cj] = int(arr[i, j])
            
    return json.dumps(out)
        

In [ ]:
confusion_matrix = np.array(
      [[ 100,   0,   0,   0,],
       [  0,  100,   0,   0,],
       [  0,   0,  100,   0,],
       [  0,   0,   0,  100,]
      ])
print(matrix_to_json(confusion_matrix))
coefficients = np.array([ 0.25, 0.25, 0.25, 0.25 ])
plot_cm(np.reshape(confusion_matrix,(*confusion_matrix.shape, 1)), 0)
plt.savefig("images/adaptive_example_cm_0.png")
plt.figure()
plt.pie(coefficients, colors=COLORS[:len(coefficients)])
plt.savefig("images/adaptive_example_pie_0.png")

In [ ]:
confusion_matrix = np.array(
      [[ 100,   0,   0,   0,],
       [  0,  100,   0,   0,],
       [  1,   0,  110,   10,],
       [  0,   0,   20,  120,]
      ])
print(matrix_to_json(confusion_matrix))
coefficients = np.array([ 0.16072842438638163, 0.1583531274742676, 0.3087885985748218, 0.3721298495645289 ])
plot_cm(np.reshape(confusion_matrix,(*confusion_matrix.shape, 1)), 0)
plt.savefig("images/adaptive_example_cm_1.png")
plt.figure()
plt.pie(coefficients, colors=COLORS[:len(coefficients)])
plt.savefig("images/adaptive_example_pie_1.png")

In [ ]:
confusion_matrix = np.array(
      [[ 100,   0,   0,   0,],
       [  0,  100,   0,   0,],
       [  1,   0,  110,   0,],
       [  0,   0,   100,  120,]
      ])

print(matrix_to_json(confusion_matrix))
coefficients = np.array([ 0.09341923607915324, 0.09203865623561897, 0.24390243902439024, 0.5706396686608376 ])
plot_cm(np.reshape(confusion_matrix,(*confusion_matrix.shape, 1)), 0)
plt.savefig("images/adaptive_example_cm_2.png")
plt.figure()
plt.pie(coefficients, colors=COLORS[:len(coefficients)])
plt.savefig("images/adaptive_example_pie_2.png")

In [ ]:
from IPython import Javascript


In [ ]:
plot_cm(np.reshape(confusion_matrix,(*confusion_matrix.shape, 1)), 0);

In [ ]:
plt.pie(coefficients, colors=COLORS[:len(coefficients)]);